In [24]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, KFold
import xgboost
from sklearn.ensemble import RandomForestRegressor 
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import minmax_scale,StandardScaler

In [2]:
os.chdir(r"C:\Users\Roozbeh\Vehicular Data\London")

In [3]:
df = pd.read_csv('AA_LONDON_DATA_VER3_DATA_TABLE.csv')

In [4]:
camdf = pd.read_csv('AA_LONDON_CAMERA_DATA_TABLE.csv')

In [5]:
df[:2]

,S_NO,CAMERA_NAME,DOWNLOAD_DATE,IMG_NAME,DENSITY_VALUE,DL_TIMESTAMP_LOCAL,COMMENTS,CAMERA_ID,DL_TIMESTAMP_CITY,SELECT_FLAG,RED,GREEN,BLUE
0,896208,549744,22-NOV-10,1290433594,16109.680,22-NOV-10 02.46.34.000000000 PM,NaN,182,22-NOV-10 01.46.34.000000000 PM,1,63621.507812,65366.414062,64108.488281
1,896209,549744,22-NOV-10,1290434044,11318.897,22-NOV-10 02.54.04.000000000 PM,NaN,182,22-NOV-10 01.54.04.000000000 PM,1,64379.992188,66520.500000,65209.515625


In [6]:
camdf[:2]

,CAMERA_ID,CAMERA_NAME,CAMERA_DESC,EASTING,NORTHING,GEO_LAT,GEO_LON,CONGESTION_ZONE
0,105,546600,A4 Cromwell Road by Earls Court Road,525351.39,178843.06,51.494572,-0.195707,0
1,106,546610,A4 Great West Road By Macbeth Street,522781.00,178369.00,51.490875,-0.232884,0


In [7]:
mergedf = df.merge(camdf,left_on="CAMERA_ID",right_on="CAMERA_ID")

tdf = mergedf[["CAMERA_ID","GEO_LON","GEO_LAT","DL_TIMESTAMP_CITY","DENSITY_VALUE"]].copy()

tdf['TIMESTAMP']  = pd.to_datetime(tdf['DL_TIMESTAMP_CITY'],format="%d-%b-%y %I.%M.%S.000000000 %p")

try:
    tdf.drop("DL_TIMESTAMP_CITY",axis=1,inplace=True)
except:
    print("column doesn't exist")

In [8]:
tdf[:2]

,CAMERA_ID,GEO_LON,GEO_LAT,DENSITY_VALUE,TIMESTAMP
0,182,-0.245122,51.616876,16109.680,2010-11-22 13:46:34
1,182,-0.245122,51.616876,11318.897,2010-11-22 13:54:04


In [9]:
tdf["TIME_EPOCH"] = (tdf.TIMESTAMP.astype(np.int64) // 10**9)

In [10]:
tdf["TIME_EPOCH"] -= tdf.TIME_EPOCH.min()

In [11]:
tdf.TIMESTAMP.describe()

count                  896246
unique                 401908
top       2010-10-24 10:17:03
freq                      163
first     2010-10-11 09:55:01
last      2010-11-22 18:59:42
Name: TIMESTAMP, dtype: object

In [12]:
tdf["TIME_BUCKET"] = tdf.TIME_EPOCH//600

In [13]:
tdf[:2]

,CAMERA_ID,GEO_LON,GEO_LAT,DENSITY_VALUE,TIMESTAMP,TIME_EPOCH,TIME_BUCKET
0,182,-0.245122,51.616876,16109.680,2010-11-22 13:46:34,3642693,6071
1,182,-0.245122,51.616876,11318.897,2010-11-22 13:54:04,3643143,6071


In [26]:
yscaler = StandardScaler()

In [27]:
yscaler.fit(tdf["DENSITY_VALUE"].values.reshape((-1,1)))

StandardScaler(copy=True, with_mean=True, with_std=True)

In [28]:
tdf.loc[:,"SCALED_DENSITY_VALUE"] = yscaler.transform(tdf["DENSITY_VALUE"].values.reshape((-1,1)))

In [29]:
tdf[:2]

,CAMERA_ID,GEO_LON,GEO_LAT,DENSITY_VALUE,TIMESTAMP,TIME_EPOCH,TIME_BUCKET,SCALED_DENSITY_VALUE
0,182,-0.245122,51.616876,16109.680,2010-11-22 13:46:34,3642693,6071,-0.459432
1,182,-0.245122,51.616876,11318.897,2010-11-22 13:54:04,3643143,6071,-0.906670


In [31]:
Xfeatures = ["GEO_LON","GEO_LAT"]
Tfeature = "TIME_EPOCH"
target = "SCALED_DENSITY_VALUE"

In [15]:
from tqdm import tqdm_notebook

In [32]:
knr = KNeighborsRegressor(weights="distance",n_jobs=-1)

In [37]:
mses_per_time_bucket = {}
r2s_per_time_bucket = {}
kf = KFold(n_splits=4,shuffle=True)
# c = 0
for t, group in tqdm_notebook(tdf.groupby("TIME_BUCKET")):
#     if c == 100:
#         break
#     c+=1
    mses = []
    r2s = []
    group = group.reset_index()
    for train_index, test_index in kf.split(group):        
        knr.fit(group.loc[train_index,Xfeatures],group.loc[train_index,target])
        r2s.append(knr.score(group.loc[test_index,Xfeatures],group.loc[test_index,target]))
        mses.append(mean_squared_error(group.loc[test_index,target],knr.predict(group.loc[test_index,Xfeatures])))
    mses_per_time_bucket[t] = np.mean(np.sqrt(mses))
    r2s_per_time_bucket[t] = np.mean(r2s)

Exception in thread Thread-28839:
Traceback (most recent call last):
  File "C:\Anaconda3\lib\threading.py", line 916, in _bootstrap_inner
    self.run()
  File "C:\Anaconda3\lib\site-packages\tqdm\_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "C:\Anaconda3\lib\_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration



In [39]:
np.mean([v for k,v in mses_per_time_bucket.items()]),np.mean([v for k,v in r2s_per_time_bucket.items()])

(0.5196335818438959, 0.47729760390096077)

In [55]:
mses_per_time_bucket_camholdout = {}
r2s_per_time_bucket_camholdout = {}
for t, group in tqdm_notebook(tdf.groupby("TIME_BUCKET")):
    try:
        test_cameras = np.random.choice(group.CAMERA_ID.unique(),size=15)

        df_train = group[~group.CAMERA_ID.isin(test_cameras)]
        df_test = group[group.CAMERA_ID.isin(test_cameras)]

        knr.fit(df_train[Xfeatures],df_train[target])

        mses_per_time_bucket_camholdout[t] = np.sqrt(mean_squared_error(df_test[target],knr.predict(df_test[Xfeatures])))
        r2s_per_time_bucket_camholdout[t] = knr.score(df_test[Xfeatures],df_test[target])
    except Exception as e:
        print("time: ",t,e)

time:  2475 Expected n_neighbors <= n_samples,  but n_samples = 3, n_neighbors = 5
time:  3066 Expected n_neighbors <= n_samples,  but n_samples = 3, n_neighbors = 5



In [56]:
np.mean([v for k,v in mses_per_time_bucket_camholdout.items()]),np.mean([v for k,v in r2s_per_time_bucket_camholdout.items()])

(0.8111102258022918, -0.37205394915957674)

### THERE IS NO POINT IN FORECAST AS IT REQUIRES TO HAVE SEEN TIME BEFORE. BASICALLY THIS MODEL IS ONLY USEFUL IN INTERPOLATION.


In [57]:
tdf.sort_values(["TIME_BUCKET"],inplace=True)

In [58]:
from sklearn.model_selection import TimeSeriesSplit

#### This below doesn't make sense as each test is within the same time bucket of 10 minute.

In [62]:
mses_per_time_bucket_forecast = {}
r2s_per_time_bucket_forecast = {}
tscv = TimeSeriesSplit(n_splits=4)
for t, group in tqdm_notebook(tdf[tdf.TIME_BUCKET < 200].groupby("TIME_BUCKET")):
    try:
        mses = []
        r2s = []
        group = group.reset_index()
        for train_index, test_index in tscv.split(group):        
            knr.fit(group.loc[train_index,Xfeatures],group.loc[train_index,target])
            r2s.append(knr.score(group.loc[test_index,Xfeatures],group.loc[test_index,target]))
            mses.append(mean_squared_error(group.loc[test_index,target],knr.predict(group.loc[test_index,Xfeatures])))
        mses_per_time_bucket_forecast[t] = np.mean(np.sqrt(mses))
        r2s_per_time_bucket_forecast[t] = np.mean(r2s)
    except Exception as e:
        print("time bucket:",t,e)

In [63]:
np.mean([v for k,v in mses_per_time_bucket_forecast.items()]),np.mean([v for k,v in r2s_per_time_bucket_forecast.items()])

(0.8789451113590842, -0.19071267846491677)